load libraries
---

In [3]:
if(!require(quantmod)) install.packages("quantmod")
library(quantmod)

if(!require(data.table)) install.packages("data.table")
library(data.table)

Loading required package: quantmod

Warning message:
"package 'quantmod' was built under R version 4.2.2"
Loading required package: xts

Warning message:
"package 'xts' was built under R version 4.2.2"
Loading required package: zoo

Warning message:
"package 'zoo' was built under R version 4.2.2"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: TTR

Warning message:
"package 'TTR' was built under R version 4.2.2"
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loading required package: data.table

Warning message:
"package 'data.table' was built under R version 4.2.2"

Attaching package: 'data.table'


The following objects are masked from 'package:xts':

    first, last




# Input data

In [4]:
# dataFull<-read.csv("D:/KTrinh/python/rquant/Kapital AMC/output data/dataFull.csv")
load("output data/dataFull.Rdata")
source("D:/KTrinh/python/rquant/Kapital AMC/clean data code/4_index.R")
marketIndex <- marketIndex[,c("date","close_VNI")]
setkey(marketIndex,date)

# Code for backtest 

## Data require

In [ ]:
dset <- dataFull[,c("ticker","date","year","quarter","month","sector","industry","subindustry","close","return","volume","exchange",
                    "Sales_net","COGS","assets","GrossProfit","Selling_expense","Managing_expense"
)][date > "2013-01-01"]

## Aglorith

In [ ]:
dset[,':='(f1 = rank(Sales_net/COGS,na.last = "keep")/.N,
            f2 = rank(Sales_net/assets,na.last = "keep")/.N,
            f3 = rank(GrossProfit/(Selling_expense + Managing_expense),na.last = "keep")/.N
    ),by=c("year","quarter","subindustry")
]

dset[,f4:= rank(f1+f2+f3,na.last = "keep")/.N
        ,by=c("year","quarter","subindustry")
        ]

dset[,':='(position=ifelse(f4>=0.9,1,0),
            weight =NA 
                )]

# code for visualize data

## split data

In [ ]:
dset2 <- dset[position==1]
dset2<-dset2[,.(re=mean(return,na.rm = TRUE)),by=date]
setkey(dset2,date)
dset2[,re1:=ifelse(is.na(re),1,1+re)][,index:=cumprod(re1)]

## Add Vnindex

In [ ]:
dset2 <- marketIndex[dset2]
dset2[,vnindex:=close_VNI/close_VNI[1]]

## plot data

In [ ]:
plot(dset2$date,dset2$index,type = "l",col="blue")    
lines(dset2$date,dset2$vnindex,col="red")

# Find value at risk Vnindex

In [ ]:
dset2[,Vnindex_return := f_ROC(vnindex,i = 10)]
hist(dset2$Vnindex_return)
mean = mean(dset2$Vnindex_return,na.rm = TRUE)
sd(dset2$Vnindex_return,na.rm = TRUE)
quantile(dset2$Vnindex_return,0.01,na.rm = TRUE)

sd(dset2[Vnindex_return<= -0.05]$Vnindex_return,na.rm = TRUE)
dset2$year=year(dset2$date)
dset2[,.(returns= mean(re,na.rm = TRUE)*250,index_return =mean(Vnindex_return,na.rm=TRUE)*25,
         stddev = sd(re,na.rm=TRUE)*sqrt(250)),by=year]
